In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [5]:
data = pd.read_csv('criteo_sampled_data.csv')

In [6]:
data.head(5)

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [7]:
cols = data.columns.values


# 数据预处理

In [8]:
# 定义特征组
dense_feats = [f for f in cols if f[0] == "I"]
sparse_feats = [f for f in cols if f[0] == "C"]

# 处理dense特征
def process_dense_feats(data, feats):
    d = data.copy()
    d = d[feats].fillna(0.0)
    for f in feats:
        d[f] = d[f].apply(lambda x: np.log(x+1) if x > -1 else -1)
    
    return d
data_dense = process_dense_feats(data, dense_feats)

# 处理sparse特征
def process_sparse_feats(data, feats):
    d = data.copy()
    d = d[feats].fillna("-1")
    for f in feats:
        label_encoder = LabelEncoder()
        d[f] = label_encoder.fit_transform(d[f])
        
    return d
data_sparse = process_sparse_feats(data, sparse_feats)

In [9]:
total_data = pd.concat([data_dense, data_sparse], axis=1)
total_data['label'] = data['label']

# 模型构建与训练

## 输入层—dense特征 

In [10]:
dense_inputs = []
for f in dense_feats:
    _input = Input([1],name=f)
    dense_inputs.append(_input)
    
    
concat_dense_inputs = Concatenate(axis=1)(dense_inputs)
concat_dense_inputs

<tf.Tensor 'concatenate/Identity:0' shape=(None, 13) dtype=float32>

## 输入层—sparse特征

In [11]:
sparse_inputs = []
for f in sparse_feats:
    _input = Input([1],name=f)
    sparse_inputs.append(_input)

In [12]:
# 对输入进行嵌入
k = 8
sparse_kd_embed = []
for i, _input in enumerate(sparse_inputs):
    f = sparse_feats[i]
    voc_size = data[f].nunique()
    reg = tf.keras.regularizers.l2(0.7)
    _embed = Embedding(voc_size+1,k,embeddings_regularizer=reg)(_input)
    _embed = Flatten()(_embed)
    sparse_kd_embed.append(_embed)

In [13]:
concat_sparse_inputs = Concatenate(axis=1)(sparse_kd_embed)
concat_sparse_inputs

<tf.Tensor 'concatenate_1/Identity:0' shape=(None, 208) dtype=float32>

## 输入层—组合sparse和dense 特征

In [14]:
embed_inputs = Concatenate(axis=1)([concat_sparse_inputs, concat_dense_inputs])
embed_inputs

<tf.Tensor 'concatenate_2/Identity:0' shape=(None, 221) dtype=float32>

## Cross Network

In [15]:
def cross_layer(x0, x1):
    """
    实现一层 cross layer
    param x0: 特征embeddings
    param x1: 第一层的输出结果
    
    """
    # 1. 获取x1层的embedding size
    embed_dim = x1.shape[-1]   # 221
    # 2. 初始化当前层的W和b
    w = tf.Variable(tf.random.truncated_normal(shape=(embed_dim,), stddev=0.01))
    b = tf.Variable(tf.zeros(shape=(embed_dim,)))
    
    # 3.计算feature crossing
    x_1w = tf.tensordot(tf.reshape(x1,[-1,1,embed_dim]),w,axes=1)
    cross = x0*x_1w
    return cross + b + x1

In [16]:
def build_cross_layer(x0,num_layer=3):
    """
    构建多层 cross layer
    param x0: 所有特征的embeddings
    param num_layers: cross net 层数
    """
    # 初始化 x1为x0
    x1 = x0
    
    # 构建多层cross net
    for i in range(num_layer):
        x1 = cross_layer(x0,x1)
        
    return x1    

In [18]:
# cross net
cross_layer_output = build_cross_layer(embed_inputs, 1)

(None, 221) Tensor("Reshape_6:0", shape=(None, 1, 221), dtype=float32) (221,)


 (None, 1)


 Tensor("mul_3:0", shape=(None, 221), dtype=float32)


## DNN部分 

In [15]:
fc_layer = Dropout(0.5)(Dense(128,activation='relu')(embed_inputs))
fc_layer = Dropout(0.3)(Dense(128,activation='relu')(fc_layer))
fc_layer_output = Dropout(0.1)(Dense(128,activation='relu')(fc_layer))

## 输出结果 

In [16]:
cross_layer_output,fc_layer_output

(<tf.Tensor 'add_5:0' shape=(None, 221) dtype=float32>,
 <tf.Tensor 'dropout_2/Identity:0' shape=(None, 128) dtype=float32>)

In [17]:
stack_layer = Concatenate()([cross_layer_output,fc_layer_output])

In [18]:
output_layer = Dense(1,activation='sigmoid',use_bias=True)(stack_layer)

## 模型编译

In [19]:
model = Model(dense_inputs+sparse_inputs, output_layer)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
C1 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
C2 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
C3 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
C4 (InputLayer)                 [(None, 1)]          0                                            
______________________________________________________________________________________________

In [21]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['binary_crossentropy',tf.keras.metrics.AUC(name='auc')])

In [22]:

train_data = total_data.loc[:500000-1]
valid_data = total_data.loc[500000:]

train_dense_x = [train_data[f].values for f in dense_feats]
train_sparse_x = [train_data[f].values for f in sparse_feats]

train_label = [train_data['label'].values]


val_dense_x = [valid_data[f].values for f in dense_feats]
val_sparse_x = [valid_data[f].values for f in sparse_feats]


val_label = [valid_data['label'].values]

In [ ]:
model.fit(train_dense_x+train_sparse_x, train_label,
          epochs=5, batch_size=128,
         validation_data=(val_dense_x+val_sparse_x, val_label),
         )

Train on 500000 samples, validate on 100000 samples
Epoch 1/5


D:\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


500000/500000 [==============================] - 202s 403us/sample - loss: 15.2771 - binary_crossentropy: 0.5138 - auc: 0.7108 - val_loss: 0.5207 - val_binary_crossentropy: 0.5129 - val_auc: 0.7187
Epoch 2/5
500000/500000 [==============================] - 201s 401us/sample - loss: 0.5361 - binary_crossentropy: 0.5049 - auc: 0.7259 - val_loss: 0.5649 - val_binary_crossentropy: 0.5099 - val_auc: 0.7245
Epoch 3/5
500000/500000 [==============================] - 206s 412us/sample - loss: 0.5792 - binary_crossentropy: 0.5019 - auc: 0.7309 - val_loss: 0.6030 - val_binary_crossentropy: 0.5073 - val_auc: 0.7287
Epoch 4/5
500000/500000 [==============================] - 198s 395us/sample - loss: 0.5932 - binary_crossentropy: 0.5002 - auc: 0.7335 - val_loss: 0.5985 - val_binary_crossentropy: 0.5051 - val_auc: 0.7318
Epoch 5/5
237312/500000 [=============>................] - ETA: 1:37 - loss: 0.5922 - binary_crossentropy: 0.4999 - auc: 0.7346- ETA: 1